<a href="https://colab.research.google.com/github/Haebuk/Python_Machine_Learning/blob/master/%EA%B5%AC%EB%82%B4%EC%8B%9D%EB%8B%B9%EC%8B%9D%EC%88%98%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [11]:
PATH = '/content/drive/MyDrive/input/235743_구내식당 식사 인원 예측 AI 경진대회_data/'
train_df = pd.read_csv(PATH + 'train.csv')
test_df = pd.read_csv(PATH + 'test.csv')

In [3]:
train_df.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


In [4]:
test_df.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,금,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치
3,2021-02-01,월,2924,108,154,538,322.0,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치
4,2021-02-02,화,2924,62,186,455,314.0,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치


In [5]:
print(train_df.shape, test_df.shape)

(1205, 12) (50, 10)


In [6]:
train_df.describe()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계
count,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000,1205.000000
mean,2807.815768,157.913693,241.142739,274.117012,43.506224,890.334440,461.772614
std,171.264404,144.190572,43.532298,246.239651,109.937400,209.505057,139.179202
min,2601.000000,23.000000,41.000000,0.000000,0.000000,296.000000,0.000000
25%,2645.000000,71.000000,217.000000,4.000000,0.000000,758.000000,406.000000
50%,2760.000000,105.000000,245.000000,299.000000,0.000000,879.000000,483.000000
75%,2962.000000,185.000000,272.000000,452.000000,0.000000,1032.000000,545.000000
max,3305.000000,1224.000000,378.000000,1044.000000,533.000000,1459.000000,905.000000


In [12]:
all_df = pd.concat([train_df, test_df], axis=0).reset_index(drop=True)

all_df['인원'] = all_df['본사정원수'] - all_df['본사휴가자수'] - all_df['본사출장자수'] - all_df['현본사소속재택근무자수']
all_df.drop(['일자', '본사휴가자수', '본사출장자수', '현본사소속재택근무자수'], axis=1, inplace=True)
all_df

,요일,본사정원수,본사시간외근무명령서승인건수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,인원
0,월,2601,238,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,2401.0
1,화,2601,319,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,2378.0
2,수,2601,111,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,2365.0
3,목,2601,355,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,2277.0
4,금,2601,34,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,2142.0
...,...,...,...,...,...,...,...,...,...
1250,월,2973,704,모닝롤/커피콩빵 우유/주스 계란후라이 누룽지탕/흑미밥 청양콩나물국 스팸구이 양상추샐...,쌀밥/흑미밥/찰현미밥 쇠고기미역국 춘천닭갈비 오지치즈후라이 가지두반장볶음 포기김치 ...,흑미밥 돈육고추장찌개 갈치구이 김치전 취나물무침 깍두기,NaN,NaN,2343.0
1251,화,2973,636,모닝롤/모닝샌드 우유/주스 계란후라이 고구마스프/흑미밥 아욱국 참치채소볶음 양상추샐...,쌀밥/귀리밥/찰현미밥 순두부백탕 매콤소갈비찜 깻잎완자전 돌나물초장무침 포기김치 시리...,추가밥 짬뽕*생면 수제찹쌀꿔바로우 메추리알곤약장조림 단무지무침 포기김치,NaN,NaN,2363.0
1252,수,2973,1,모닝롤/호떡맥모닝 우유/주스 계란후라이 팥죽/흑미밥 닭살해장국 우엉채조림 양상추샐러...,쌀밥/흑미밥/찰현미밥 냉이국 돈육간장불고기 비빔냉면 오이나물볶음 겉절이김치 양상추샐...,단호박카레라이스 시금치된장국 소떡소떡 파프리카해초무침 감귤쥬스 포기김치,NaN,NaN,2329.0
1253,목,2973,509,모닝롤/크로크무슈 우유/주스 계란후라이 누룽지탕/흑미밥 감자국 두부양념조림 양상추샐...,쌀밥/옥수수밥/찰현미밥 맑은떡국 (New)로제찜닭 가자미구이*장 유채나물무침 포기김...,흑미밥 어묵매운탕 쇠고기숙주볶음 채소계란찜 쑥갓생무침 김치볶음,NaN,NaN,2306.0


In [13]:
regex = "\((.*?)\)"
for i in range(len(all_df)):
    all_df['조식메뉴'][i] = re.sub(regex, '', all_df['조식메뉴'][i])
    all_df['중식메뉴'][i] = re.sub(regex, '', all_df['중식메뉴'][i])
    all_df['석식메뉴'][i] = re.sub(regex, '', all_df['석식메뉴'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [14]:
all_df

,요일,본사정원수,본사시간외근무명령서승인건수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,인원
0,월,2601,238,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 된장찌개 쥐어채무침 포...,쌀밥/잡곡밥 오징어찌개 쇠불고기 계란찜 청포묵무침 요구르트 포기김치,쌀밥/잡곡밥 육개장 자반고등어구이 두부조림 건파래무침 포기김치,1039.0,331.0,2401.0
1,화,2601,319,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 호박젓국찌개 시래기조...,쌀밥/잡곡밥 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무침 요구르트 배추겉...,콩나물밥*양념장 어묵국 유산슬 아삭고추무침 바나나 포기김치,867.0,560.0,2378.0
2,수,2601,111,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 콩나물국 느타리호박...,카레덮밥 팽이장국 치킨핑거 쫄면야채무침 견과류조림 요구르트 포기김치,쌀밥/잡곡밥 청국장찌개 황태양념구이 고기전 새송이버섯볶음 포기김치,1017.0,573.0,2365.0
3,목,2601,355,모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 근대국 멸치볶음 포...,쌀밥/잡곡밥 쇠고기무국 주꾸미볶음 부추전 시금치나물 요구르트 포기김치,미니김밥*겨자장 우동 멕시칸샐러드 군고구마 무피클 포기김치,978.0,525.0,2277.0
4,금,2601,34,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 재첩국 방풍나물 포기...,쌀밥/잡곡밥 떡국 돈육씨앗강정 우엉잡채 청경채무침 요구르트 포기김치,쌀밥/잡곡밥 차돌박이찌개 닭갈비 감자소세지볶음 콩나물무침 포기김치,925.0,330.0,2142.0
...,...,...,...,...,...,...,...,...,...
1250,월,2973,704,모닝롤/커피콩빵 우유/주스 계란후라이 누룽지탕/흑미밥 청양콩나물국 스팸구이 양상추샐...,쌀밥/흑미밥/찰현미밥 쇠고기미역국 춘천닭갈비 오지치즈후라이 가지두반장볶음 포기김치 ...,흑미밥 돈육고추장찌개 갈치구이 김치전 취나물무침 깍두기,NaN,NaN,2343.0
1251,화,2973,636,모닝롤/모닝샌드 우유/주스 계란후라이 고구마스프/흑미밥 아욱국 참치채소볶음 양상추샐...,쌀밥/귀리밥/찰현미밥 순두부백탕 매콤소갈비찜 깻잎완자전 돌나물초장무침 포기김치 시리...,추가밥 짬뽕*생면 수제찹쌀꿔바로우 메추리알곤약장조림 단무지무침 포기김치,NaN,NaN,2363.0
1252,수,2973,1,모닝롤/호떡맥모닝 우유/주스 계란후라이 팥죽/흑미밥 닭살해장국 우엉채조림 양상추샐러...,쌀밥/흑미밥/찰현미밥 냉이국 돈육간장불고기 비빔냉면 오이나물볶음 겉절이김치 양상추샐...,단호박카레라이스 시금치된장국 소떡소떡 파프리카해초무침 감귤쥬스 포기김치,NaN,NaN,2329.0
1253,목,2973,509,모닝롤/크로크무슈 우유/주스 계란후라이 누룽지탕/흑미밥 감자국 두부양념조림 양상추샐...,쌀밥/옥수수밥/찰현미밥 맑은떡국 로제찜닭 가자미구이*장 유채나물무침 포기김치 양상추...,흑미밥 어묵매운탕 쇠고기숙주볶음 채소계란찜 쑥갓생무침 김치볶음,NaN,NaN,2306.0


In [15]:
col = ['조식메뉴', '중식메뉴', '석식메뉴']
for i in col:
    all_df[i] = all_df[i].str.split()
all_df

,요일,본사정원수,본사시간외근무명령서승인건수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,인원
0,월,2601,238,"[모닝롤/찐빵, 우유/두유/주스, 계란후라이, 호두죽/쌀밥, 된장찌개, 쥐어채무침,...","[쌀밥/잡곡밥, 오징어찌개, 쇠불고기, 계란찜, 청포묵무침, 요구르트, 포기김치]","[쌀밥/잡곡밥, 육개장, 자반고등어구이, 두부조림, 건파래무침, 포기김치]",1039.0,331.0,2401.0
1,화,2601,319,"[모닝롤/단호박샌드, 우유/두유/주스, 계란후라이, 팥죽/쌀밥, 호박젓국찌개, 시래...","[쌀밥/잡곡밥, 김치찌개, 가자미튀김, 모둠소세지구이, 마늘쫑무침, 요구르트, 배추...","[콩나물밥*양념장, 어묵국, 유산슬, 아삭고추무침, 바나나, 포기김치]",867.0,560.0,2378.0
2,수,2601,111,"[모닝롤/베이글, 우유/두유/주스, 계란후라이, 표고버섯죽/쌀밥, 콩나물국, 느타리...","[카레덮밥, 팽이장국, 치킨핑거, 쫄면야채무침, 견과류조림, 요구르트, 포기김치]","[쌀밥/잡곡밥, 청국장찌개, 황태양념구이, 고기전, 새송이버섯볶음, 포기김치]",1017.0,573.0,2365.0
3,목,2601,355,"[모닝롤/토마토샌드, 우유/두유/주스, 계란후라이, 닭죽/쌀밥, 근대국, 멸치볶음,...","[쌀밥/잡곡밥, 쇠고기무국, 주꾸미볶음, 부추전, 시금치나물, 요구르트, 포기김치]","[미니김밥*겨자장, 우동, 멕시칸샐러드, 군고구마, 무피클, 포기김치]",978.0,525.0,2277.0
4,금,2601,34,"[모닝롤/와플, 우유/두유/주스, 계란후라이, 쇠고기죽/쌀밥, 재첩국, 방풍나물, ...","[쌀밥/잡곡밥, 떡국, 돈육씨앗강정, 우엉잡채, 청경채무침, 요구르트, 포기김치]","[쌀밥/잡곡밥, 차돌박이찌개, 닭갈비, 감자소세지볶음, 콩나물무침, 포기김치]",925.0,330.0,2142.0
...,...,...,...,...,...,...,...,...,...
1250,월,2973,704,"[모닝롤/커피콩빵, 우유/주스, 계란후라이, 누룽지탕/흑미밥, 청양콩나물국, 스팸구...","[쌀밥/흑미밥/찰현미밥, 쇠고기미역국, 춘천닭갈비, 오지치즈후라이, 가지두반장볶음,...","[흑미밥, 돈육고추장찌개, 갈치구이, 김치전, 취나물무침, 깍두기]",NaN,NaN,2343.0
1251,화,2973,636,"[모닝롤/모닝샌드, 우유/주스, 계란후라이, 고구마스프/흑미밥, 아욱국, 참치채소볶...","[쌀밥/귀리밥/찰현미밥, 순두부백탕, 매콤소갈비찜, 깻잎완자전, 돌나물초장무침, 포...","[추가밥, 짬뽕*생면, 수제찹쌀꿔바로우, 메추리알곤약장조림, 단무지무침, 포기김치]",NaN,NaN,2363.0
1252,수,2973,1,"[모닝롤/호떡맥모닝, 우유/주스, 계란후라이, 팥죽/흑미밥, 닭살해장국, 우엉채조림...","[쌀밥/흑미밥/찰현미밥, 냉이국, 돈육간장불고기, 비빔냉면, 오이나물볶음, 겉절이김...","[단호박카레라이스, 시금치된장국, 소떡소떡, 파프리카해초무침, 감귤쥬스, 포기김치]",NaN,NaN,2329.0
1253,목,2973,509,"[모닝롤/크로크무슈, 우유/주스, 계란후라이, 누룽지탕/흑미밥, 감자국, 두부양념조...","[쌀밥/옥수수밥/찰현미밥, 맑은떡국, 로제찜닭, 가자미구이*장, 유채나물무침, 포기...","[흑미밥, 어묵매운탕, 쇠고기숙주볶음, 채소계란찜, 쑥갓생무침, 김치볶음]",NaN,NaN,2306.0


- 메뉴 정리

    - 리스트에 담겨있는 조/중/석식별 메뉴를 카운트
    - TF/IDF를 쓸까?

In [22]:
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
list1 = ['a','b','c','d']
list2 = ['a','c','d']
tfidf.fit_transform(all_df['조식메뉴'].values)
tfidf


AttributeError: ignored

In [28]:
tfidf = TfidfVectorizer()

list_contents =[]
for line in all_df['조식메뉴']:
    print(line)

# this matrix will have only 3 columns because we have forced
# the vectorizer to use just the words foo bar and baz
# so it'll ignore all other words in the documents.
#tfidf_matrix = tfidf_vectorizer.fit_transform(list_contents) 

['모닝롤/찐빵', '우유/두유/주스', '계란후라이', '호두죽/쌀밥', '된장찌개', '쥐어채무침', '포기김치']
['모닝롤/단호박샌드', '우유/두유/주스', '계란후라이', '팥죽/쌀밥', '호박젓국찌개', '시래기조림', '포기김치']
['모닝롤/베이글', '우유/두유/주스', '계란후라이', '표고버섯죽/쌀밥', '콩나물국', '느타리호박볶음', '포기김치']
['모닝롤/토마토샌드', '우유/두유/주스', '계란후라이', '닭죽/쌀밥', '근대국', '멸치볶음', '포기김치']
['모닝롤/와플', '우유/두유/주스', '계란후라이', '쇠고기죽/쌀밥', '재첩국', '방풍나물', '포기김치']
['팬케익/찐빵', '우유/두유/주스', '계란후라이', '견과류죽/쌀밥', '감자찌개', '명엽채무침', '포기김치']
['모닝롤/야채샌드', '우유/두유/주스', '계란후라이', '고구마죽/쌀밥', '봄동된장국', '숙주나물', '포기김치']
['모닝롤/치즈프레즐', '우유/두유/주스', '계란후라이', '잣죽/쌀밥', '민물새우찌개', '콩조림', '포기김치']
['모닝롤/마늘빵', '우유/두유/주스', '계란후라이', '단호박죽/쌀밥', '어묵국', '김구이', '포기김치']
['모닝롤/참치샌드', '우유/두유/주스', '계란후라이', '흑임자죽/쌀밥', '북어계란국', '무생채', '포기김치']
['모닝롤/인절미토스트', '우유/두유/주스', '계란후라이', '계란후라이', '선지해장국', '톳두부무침', '포기김치']
['모닝롤/씨크립샌드', '우유/두유/주스', '계란후라이', '밤죽/쌀밥', '홍합탕', '오이생채', '포기김치']
['모닝롤/팬케익', '우유/주스', '계란후라이', '브로컬리죽/쌀밥', '고추장찌개', '시금치나물', '포기김치']
['모닝롤/야채샌드', '우유/주스', '계란후라이', '닭죽/쌀밥', '재첩국', '참나물땅콩가루무침', '포기김치']
['모닝롤/꿀호떡', '우유/주스', '계란후라이', '김치죽/쌀밥', '